## Méthodes d'optimisation stochastique

## I. Minimisation stochastique d'une fonction déterministe


On s'intéresse au problème
$$\min f(x) =\max_{i=1\dots m}|a_i^Tx-b_i |.$$
On suppose que les $a_i$ sont les lignes $i$ d'une matrice $A$ de taille $100 \times 20$ , et les $b_i$ sont les lignes d'un second membre $b$. Ces quantités sont générées une fois pour toutes à partir de distributions Gaussiennes de moyenne nulle et d'écart type identité.

**Question 1 :** Constuire $A$ et $b$. Proposer le calcul d'un sous-gradient en $x$ de $f$.


In [12]:
# Julia 1.5.2

In [87]:
#import Pkg
#Pkg.add("Distributions")
using Distributions
using LinearAlgebra

In [118]:
# Construction des données A  et b
# Insérer votre code
m = 100
n = 20
A = rand(Normal(0,1),(m,n))
b = rand(Normal(0,1),(m,1))
x = zeros(n,1)

f(x) = maximum(abs.(A*x-b))
# Fin insérer code

# Fonction calculant un sous-gradient en x de f
function subgrad(A,b,x) 
    # Insérer votre code
    fct = A*x-b
    (~,i) = findmax(abs.(fct))
    j = i[1]
    g = A[j,:]*sign(fct[j])

    return g
    # Fin insérer code
end

#print((A*x-b))
#print(f(x))
#subgrad(A,b,x)

subgrad (generic function with 1 method)

**Question 2 :** Ce problème peut se reformuler comme un problème de programmation linéaire : $$(\mathcal{P}_{lp})\quad \left\{ \begin{array}{c} \min_{(x,R)\in \mathbb{R}^n\times \mathbb{R}} h(x,R)=R\\
s.c. \quad-R*e\leq A*x-b\leq R*e\end{array}\right.$$ avec $e=[1,\cdots,1]^T\in \mathbb{R}^m$. Résoudre le problème $(\mathcal{P}_{lp})$ en utilisant le solveur "GLPK" de la librairie JuMP. Plus d'informations sont disponibles ici :  http://www.juliaopt.org/JuMP.jl/latest/quickstart/

In [119]:
#using Pkg
#Pkg.add("JuMP"); 
#Pkg.add("GLPK")
using JuMP
using GLPK
e = ones(100,1)

# Définition du modèle
# Insérer votre code
model = Model(GLPK.Optimizer)
# Fin insérer code

# Définition des variables d'optimisation
# Insérer votre code
@variable(model, x[1:n])
@variable(model, R)
# Fin insérer code

# Définition de la fonctionnelle à minimiser
#Insérer votre code
@objective(model, Min, R)
# Fin insérer code

# Définition des contraintes
# Insérer votre code
for i in 1:m
    @constraint(model, sum(A[i,j]*x[j] for j=1:n) - b[i] >= - R);
    @constraint(model, sum(A[i,j]*x[j] for j=1:n) - b[i] <= R);
end
# Fin insérer code
        
# Résolution        
# Insérer votre code
optimize!(model)
# Fin insérer code
        
# Résultats à optimalité                
# Insérer votre code
#@show objective_value(model)
#@show JuMP.value.(x)
fstar = objective_value(model);
xstar = JuMP.value.(x);
Rstar = JuMP.value.(R);
# Fin insérer code
println("The function value at the solution is: ",Rstar, " or ",findmax(abs.(A*xstar-b)))


The function value at the solution is: 1.6696526654192467 or (1.6696526654192487, CartesianIndex(49, 1))


**Question 3 :** Résoudre le problème en utilisant un algorithme de sous-gradient. Dans un premier temps vous utiliserez un sous-gradient exact (Question 1), puis vous introduirez un bruit artificiel qui suit une distribution normale de moyenne nulle et d'écart-type $3 e-1$.

**Question 4 :** Vous afficherez les courbes de convergence de $f_{best}^k-f_{star}$, avec $f_{star}$ obtenue à la Question 1. 

In [120]:
#Pkg.add("Plots")
using Plots

#Initialisation
x = zeros(n,1);
xp = x
i = 0;
fbest =1e10; # $f_{best}^0$: cas du sous-gradient exact
fbestp=1e10; # $f_{best}^0$: cas du sous-gradient bruité
histo =[];# Suite des itérés f_{best}^k-f_{star} pour le cas du sous-gradient exact
histop=[];# Suite des itérés f_{best}^k-f_{star}, pour le cas du sous-gradient bruité

#Niveau de bruit
noise_lvl=.3;

# Resolution 
# Insérer votre code
for i in 1:100
    # Definition de alpha
    alpha = 1/(100*i)
    
    # Calcul du sous-gradient 
    deltaf = subgrad(A,b,x)
    deltafp = subgrad(A,b,xp) + noise_lvl*rand(n,1)
    
    # Mise à jour des x
    x = x - alpha*deltaf
    xp = xp - alpha*deltafp
    
    # Mise à jour des fbest
    fbest = min(fbest,findmax(abs.(A*x-b))[1])
    fbestp = min(fbestp,findmax(abs.(A*xp-b))[1])
    
    append!(histo,fbest-f_star)
    append!(histop,fbestp-f_star)
end
# Fin insérer code

#Affichage des courbes de convergence
plotly();
iter=1:100;
hf=[histo,histop];
plot(iter,hf,title="Convergence curves",label=["Exact" "Noisy"],lw=3)

<!DOCTYPE html>
 
 
 Plots.jl

## II. Minimisation stochastique d'une fonction stochatique



On s'intéresse au problème
$$\min_x f(x) =\text{E} (\max_{i=1\dots m}|a_i^Tx-b_i |).$$
On suppose que les $a_i$ sont les lignes $i$ d'une matrice $A$ de taille $100 \times 20$ , et les $b_i$ sont les lignes d'un second membre $b$. Ces quantités sont générées une fois pour toutes à partir de distributions Gaussiennes de moyenne connue $\bar{A}$ et $\bar{b}$ (non nécesairement nulle) et d'écart type identité.

**Question 5 :** Proposer deux fonctions d'évaluation de la fonction $f$ et d'un sous-gradient de $f$ basées sur des échantillons de taille $M$.


In [121]:
# Evaluation de f
function fvals(Abar,bbar,noise,xs,M) 
    # Abar, bbar : moyenne des données
    # noise : niveau de bruit 
    # xs : vecteur courant 
    # M: taille de l'échantillon

# Insérer votre code
    s = 0
    m,n = size(Abar)
    for i = 1:M
        f = findmax(abs.((Abar + noise*randn(m,n))*xs - (bbar + noise*randn(m,1))))[1];
        s += f;
    end
    s = (1/M)*s;
    return s
# Fin insérer code

end

# Evaluation d'un sous-gradient
function subgrads(Abar,bbar,noise,xs,M)
    # Abar, bbar : moyenne des données
    # noise : niveau de bruit 
    # xs : vecteur courant 
    # M: taille de l'échantillon
    
# Insérer votre code
    m,n = size(Abar);
    s = zeros(n,1);
    for i = 1:M
        #A1 = Abar + noise*randn(m,n)
        #b1 = bbar + noise*randn(m,1)
        s += subgrad(Abar + noise*randn(m,n), bbar + noise*randn(m,1), xs)
    end
    s = (1/M)*s
    return s
# Fin insérer code

end


subgrads (generic function with 1 method)

**Question 6 :** Comparer les courbes de convergence du problème déterministe $$ \min_x f(x) = \max_{i=1\dots m}|\text{E} (a_i)^Tx-\text{E} (b_i) |,$$ et du problème stochastique obtenu avec $M=10,100,1000$ échantillons.

In [127]:
# Données
Abar=10*ones(m,n)+1*randn(m,n);
bbar=10*randn(m,1);

#x_0
xd = zeros(n,1); # problème déterministe
xs = xd; # problème stochastique

# Bruit et echantillon
M  = 100;
noise  = 4;

#j = 0;

fbestd =1e10; # $f_{best}^0$: cas d'une résolution déterministe
fbests =1e10; # $f_{best}^0$: cas d'une résolution stochastique
histod =[]; # Suite des itérés f_{best}^k pour le cas d'une résolution déterministe
histos =[]; # Suite des itérés f_{best}^k pour le cas d'une résolution stochastique

# Insérer votre code
for i in 1:100
    # Definition de alpha
    alpha = 1/i
    
    # Calcul du sous-gradient 
    deltafd = subgrad(Abar,bbar,xd)
    deltafs = subgrads(Abar,bbar,noise,xs,M)
    
    # Mise à jour des x
    xd = xd - alpha*deltafd
    xs = xd - alpha*deltafs
    
    # Mise à jour des fbest
    fbestd = min(fbestd,findmax(abs.(Abar*xd-bbar))[1])
    fbests = min(fbests,fvals(Abar,bbar,noise,xs,M))
    append!(histod,fbestd)
    append!(histos,fbests)
end

# Fin insérer code
#Affichage
#plotly();
iter=1:100;
hf=[histod,histos];
plot(iter,hf,title="Convergence curves",label=["Deterministic" "Stochastic"],lw=3)

<!DOCTYPE html>
 
 
 Plots.jl

**Question 7 :** Répéter les expériences et comparer les valeurs meilleurs valeurs de f obtenues ($f_{best}$) aprs un nombre fixé d'itérations. 

In [129]:
# Données
Abar=A;
bbar=b;

# x_0
xd = zeros(n,1); # résolution déterministe
xs = xd;  # résolution stochastique

# Bruit et echantillon
M  = 200;
noise  = 4;

#Nombre d'itérations
niter=100;

j = 0;

fbestd =1e10; # $f_{best}^0$: cas d'une résolution déterministe
fbests =1e10; # $f_{best}^0$: cas d'une résolution stochastique
fbesttd=[]; # f_{best} pour chaque expériences dans le cas d'une résolution déterministe
fbestts=[]; # f_{best} pour chaque expériences dans le cas d'une résolution stochastique

for nexp=1:20
    # Répétition des expériences    
# Insérer votre code
    for i in 1:niter
        
        # Definition de alpha
        alpha = 1/i

        # Calcul du sous-gradient 
        deltafd = subgrad(Abar,bbar,xd)
        deltafs = subgrads(Abar,bbar,noise,xs,M)

        # Mise à jour des x
        xd = xd - alpha*deltafd
        xs = xd - alpha*deltafs

        # Mise à jour des fbest
        fbestd = min(fbestd,findmax(abs.(Abar*xd-bbar))[1])
        fbests = min(fbests,fvals(Abar,bbar,noise,xs,M))
    end
# Fin insérer code
    append!(fbesttd,fbestd)
    append!(fbestts,fbests)
end

#Affichage
#plotly();
iter=1:20;
hf=[fbesttd,fbestts];
plot(iter,hf,title="Convergence curves",label=["Deterministic" "Stochastic"],lw=3)

<!DOCTYPE html>
 
 
 Plots.jl